In [ ]:
!pip install pdfminer.six
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 5.7 MB/s eta 0:00:00


In [ ]:
import pypdf
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTLine, LTFigure
from PIL import Image
import matplotlib.pyplot as plt
import os

import cv2
from imutils.perspective import four_point_transform
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
# Отображение картинки pyplot
def plt_img_show(img):
    fig = plt.figure(dpi=4)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [ ]:
# Отображение картинки cv2
def cv2_show(name, img):
#  cv2.imshow(name, img) # Для компа
    cv2_imshow(img) # Для колаба
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Классы Голосования

In [ ]:
class Point:
    'Точка с координатами'
    def __init__(self, x:float, y:float):
        self.x: float = round(x,2)
        self.y: float = round(y,2)

#____________________________________________________________________
class Marker:
    'Маркер'
    def __init__(self, name: str, x0: float, y0: float, x1: float, y1: float):
        self.name: str = name
        self.cornerLD: Point = Point(x0, y0)
        self.cornerRUp: Point = Point(x1, y1)

    def getCorners(self):
        return [self.cornerLD.x, self.cornerLD.y], [self.cornerRUp.x, self.cornerRUp.y]
#____________________________________________________________________
class WordData:
    'Данные слова голосования'
    def __init__(self, word: str, cornerLD: Point, cornerRUp: Point):
        self.word: str = word
        self.cornerLD: Point = cornerLD
        self.cornerRUp: Point = cornerRUp

    def getCorners(self):
        return [self.cornerLD.x, self.cornerLD.y], [self.cornerRUp.x, self.cornerRUp.y]

#____________________________________________________________________
class QuestionLine:
    'Полоса данных по вопросу'
    def __init__(self, question: str):
        self.question: str = question # Номер вопроса
        self.words: dict[int, WordData] = {}

    def addWord(self, wordLineNumber: int, word: str, cornerLD: Point, cornerRUp: Point):
        self.words[wordLineNumber] = WordData(word, cornerLD, cornerRUp)

#____________________________________________________________________
class BulletinNumber:
    'Номер голосования'
    def __init__(self, value: str, x0: float, y0: float, x1: float, y1: float):
        self.value: str = value
        self.cornerLD: Point = Point(x0, y0)
        self.cornerRUp: Point = Point(x1, y1)

    def getCorners(self):
        return [self.cornerLD.x, self.cornerLD.y], [self.cornerRUp.x, self.cornerRUp.y]

#____________________________________________________________________
class Footer:
    'Нижний колонтитул'
    def __init__(self, pageNumber: str, bulletinNumber: str, x0: float, y0: float, x1: float, y1: float):
        self.pageNumber: str = pageNumber
        self.bulletinNumber: str = bulletinNumber
        self.cornerLD: Point = Point(x0, y0)
        self.cornerRUp: Point = Point(x1, y1)

    def getCorners(self):
        return [self.cornerLD.x, self.cornerLD.y], [self.cornerRUp.x, self.cornerRUp.y]

#____________________________________________________________________
class Page:
    'Страница'
    def __init__(self):
        self.cornerLD: Point
        self.cornerRD: Point
        self.cornerLUp: Point
        self.cornerRUp: Point
        self.images = {}
        self.__lineIdDict: dict[int, int] = {} # Словарь id номеров строк голосования lineId:number
        self.questionLines: dict[int, QuestionLine] = {}
        self.footer: Footer
        self.markers: dict[int, Marker] = {}

    def setCorners(self, cornerLD: Point, cornerRUp: Point):
        self.cornerLD = cornerLD
        self.cornerRUp = cornerRUp
        self.cornerRD = Point(cornerRUp.x, cornerLD.y)
        self.cornerLUp = Point(cornerLD.x, cornerRUp.y)

    def setCoordinates(self, x0: float, y0: float, x1: float, y1: float):
        self.cornerLD = Point(x0, y0)
        self.cornerRUp = Point(x1, y1)
        self.cornerRD = Point(x1, y0)
        self.cornerLUp = Point(x0, y1)

    def setImages(self, imageName: str, image):
        self.images[imageName] = image

    def setQuestion(self, lineId: int, question: str, wordLineNumber: int, word: str, cornerLD: Point, cornerRUp: Point):
        if lineId not in self.__lineIdDict:
            self.__lineIdDict[lineId] = len(self.__lineIdDict)
            self.questionLines[self.__lineIdDict[lineId]] = QuestionLine(question=question)
        self.questionLines[self.__lineIdDict[lineId]].addWord(wordLineNumber=wordLineNumber, word=word, cornerLD=cornerLD, cornerRUp=cornerRUp)

    def setFooter(self, pageNumber: str, bulletinNumber: str, x0: float, y0: float, x1: float, y1: float):
        self.footer = Footer(pageNumber, bulletinNumber, x0, y0, x1, y1)

    def setMarker(self, markerName: str, x0: float, y0: float, x1: float, y1: float):
        self.markers[len(self.markers)] = Marker(markerName, x0, y0, x1, y1)

    def getCorners(self):
        return [[self.cornerLD.x, self.cornerLD.y], [self.cornerRUp.x, self.cornerRUp.y]]

    def getImages(self):
        result = []
        for key, img in self.images.items():
            result.append([key, img])
        return result

    def getQuestionLines(self):
        result = []
        for key, questionLine in self.questionLines.items():
            result.append([questionLine.question, questionLine.words])
        return result

    def getMarkers(self):
        result = []
        for key, marker in self.markers.items():
            img=None
            for ikey, image in self.images.items():
                if marker.name == ikey.rsplit(".", 1)[0]:
                    img=image
                    break
            result.append([key, marker, img])
        return result
#____________________________________________________________________
class Bulletin:
    'Бюллетень голосования'
    def __init__(self, fileName, pageCount):
        self.fileName: str = fileName
        self.bulletinNumber: BulletinNumber
        self.pages: list[Page] = []
        for i in range(pageCount):
            self.pages.append(Page())

    def setBulletinNumber(self, value: str, x0: float, y0: float, x1: float, y1: float):
        self.bulletinNumber = BulletinNumber(value, x0, y0, x1, y1)

    def setPageCorners(self, pageNum: int, cornerLD: Point, cornerRUp: Point):
        self.pages[pageNum].setCorners(cornerLD, cornerRUp)

    def setPageCoordinates(self, pageNum: int, x0: float, y0: float, x1: float, y1: float):
        self.pages[pageNum].setCoordinates(x0, y0, x1, y1)

    def setPageImage(self, pageNum: int, imageName: str, image):
        self.pages[pageNum].setImages(imageName, image)

    def setPageQuestion(self, pageNum: int, lineId: int, question:str, wordLineNumber: int, word: str, x0: float, y0: float, x1: float, y1: float):
        self.pages[pageNum].setQuestion(lineId=lineId, question=question, wordLineNumber=wordLineNumber, word=word, cornerLD=Point(x0, y0), cornerRUp=Point(x1, y1))

    def setPageFooter(self, pageNum: int, pageNumber: str, bulletinNumber: str, x0: float, y0: float, x1: float, y1: float):
        self.pages[pageNum].setFooter(pageNumber, bulletinNumber, x0, y0, x1, y1)

    def getCorners(self):
        result = []
        for page_num, page in enumerate(self.pages):
            result.append([page_num, page.getCorners()])
        return result

    def getBullotNumber(self):
        return self.bulletinNumber

    def getData(self):
        result = []
        for page_num, page in enumerate(self.pages):
            result.append([page_num, page.getCorners(), page.getImages(), page.getMarkers(), page.getQuestionLines(), page.footer])
        return result

#____________________________________________________________________
class Meeting:
    'Голосование'

    wordsOfVoices = ['ЗА','ПРОТИВ', 'ВОЗДЕРЖАЛСЯ'] # список слов голосования
    wordsVLines: dict[int, list[int]] = {0: [0,3], 1:[3, 6], 2:[6, 8]} # номера вертикальных линий для определения бокса каждого слова
    line_pixels_range = 10 # Диапазон по у, в пределах которого слова голосования wordsOfVoices принимаются находящимися в одной строке

    def getData(self):
        return self.baseBulletin.getData()

    def getQuestionLines(self, pageElements, wordIndex: int, yMax:float, yMin: float):
        vLines: dict[int, list[float]] = {}
        ln_pixels_range = 4  # Диапазон по x, в пределах которого вертикальные линии таблиц голосования lines принимаются одинаковыми
        # Перебираем элементы, составляющие страницу сверху вниз
        for i, component in enumerate(pageElements):
            # Извлекаем элемент структуры страницы
            element = component[1]
            y = element.y1
            if (y>yMin and y<yMax) and isinstance(element, LTLine): # Ищем линии в диапазоне от вопроса голосования до строки голосования
                p0=Point(element.x0, element.y0)
                p1=Point(element.x1, element.y1)
                if abs(p1.x-p0.x) < 1: # Если координаты x элемента равны (вертикальная линия)
                    key = int(p0.x/ln_pixels_range+0.5)
                    line = vLines.get(key, None)
                    x=p0.x
                    if line is not None:
                        x = Point((line[0]+x)/2,0).x # Округляем через Point
                    vLines[key] = [x, p0.y, p1.y]
        # Отсортируем словарь по ключу
        vLines = dict(sorted(vLines.items()))
        # заменим ключи на порядковые номера
        count = len(vLines)
        key_list = list(range(count))
        vLines = dict(zip(key_list, list(vLines.values())))
        return [int(yMin), vLines]

    def __init__(self, baseFileName):
        # создаём объект файла PDF
        pdfFile = open(baseFileName, 'rb')

        # Используем pypdf для определения размеров страницы и извлечения образцов картинок
        # создаём объект считывателя PDF
        pdfReaded = pypdf.PdfReader(pdfFile)
        pages = pdfReaded.pages
        self.baseBulletin: Bulletin = Bulletin(baseFileName, len(pages))
        for pageNum, page in enumerate(pages) :
            #Определяем координаты углов страницы
            pageBox = page.mediabox
            self.baseBulletin.setPageCoordinates(pageNum = pageNum, x0 = pageBox[0], y0 = pageBox[1], x1 = pageBox[2], y1 = pageBox[3])

            # Определяем имеющиеся на странице картинки
            for j, img in enumerate(page.images):
                self.baseBulletin.setPageImage(pageNum=pageNum, imageName = img.name, image=img.image) # если на странице может быть несколько разных картинок с одинаковым именем (не уверен), то будет только последняя

        # Используем pdfminer.six для работы с объектами PDF
        prevTextData = {'text':'_', 'y':None} # Задаем словарь для хранения предыдущего текста и его координаты y
        vLines = None # Переменная для промежуточного хранения словаря вертикальных линий строки
        for pageNum, page in enumerate(extract_pages(baseFileName)):
            # Находим все элементы, добавляем координату y1 и элемент
            pageElements = [(element.y1, element) for element in page._objs]
            # Сортируем все элементы по порядку нахождения на странице
            pageElements.sort(key=lambda a: a[0], reverse=True) # Сортировка по убыванию, так как y=0 - нижний край страницы
            # Определяем высоту страницы
            pageHeigth = self.baseBulletin.pages[pageNum].getCorners()[1][1]-self.baseBulletin.pages[pageNum].getCorners()[0][1]

            # Перебираем элементы, составляющие страницу сверху вниз
            for i, component in enumerate(pageElements):
                # Извлекаем элемент структуры страницы
                element = component[1]

                if element.y1>800 or element.y1<50 and isinstance(element, LTFigure):
                    name = element.name
                    pageImages = self.baseBulletin.pages[pageNum].getImages()
                    for i, image_data in enumerate(pageImages):
                        if name == image_data[0].rsplit(".", 1)[0]:
                            box = element.bbox
                            self.baseBulletin.pages[pageNum].setMarker(markerName=name, x0=box[0], y0=box[1], x1=box[2], y1=box[3])
                            break
                    #print(element.name, element.bbox) [image_left, image_top, image_right, image_bottom] = [element.x0,element.y0,element.x1,element.y1]

                if isinstance(element, LTTextContainer):
                    text = ''.join(c for c in element.get_text() if c.isprintable())
                    if element.y1>750 and ' №' in text:
                        texts = text.split(sep='№')
                        value = '' if len(texts)<2 else texts[1]
                        self.baseBulletin.setBulletinNumber(value=value, x0=element.x0, y0=element.y0, x1=element.x1, y1=element.y1)
                    elif element.y1>100: # Не включаем тексты, координаты которых заведомо выходят за пределы размещения текстовых блоков с вопросами
                        try:
                            index = self.wordsOfVoices.index(text) # Определяем индекс слова из списка слов голосования
                            line_id = int(element.y0/self.line_pixels_range + 0.5) # Определяем id строки с учетом координаты y в пределах +-line_pixels_range/2
                            if vLines is None: # Если линии еще не считывали
                                if prevTextData['y']>element.y0:
                                    yMax = prevTextData['y']
                                else:
                                    yMax = self.baseBulletin.pages[pageNum].getCorners()[1][1]
                                vLines = self.getQuestionLines(pageElements=pageElements, wordIndex=index, yMax=yMax, yMin=element.y0) # Находим вертикальные линии таблицы строки голосования
                            x0=y0=x1=y1=0
                            if abs(element.y0-vLines[0])<self.line_pixels_range:
                                left, right = self.wordsVLines.get(index, [0,1])
                                x0=vLines[1][left][0]
                                y0=pageHeigth - vLines[1][left][1] # Координаты считаются снизу вверх, поэтому вычитаем их из высоты листа
                                x1=vLines[1][right][0]
                                y1=pageHeigth - vLines[1][right][2]# Координаты считаются снизу вверх, поэтому вычитаем их из высоты листа
                            self.baseBulletin.setPageQuestion(
                                pageNum=pageNum, lineId=line_id, question=prevTextData['text'],
                                    wordLineNumber=index, word=text, x0=x0, y0=y0, x1=x1, y1=y1)
                        except ValueError:
                            # Сохраняем текст и координату y для последующих итераций (когда будет найдена строка голосования)
                            prevTextData['text'] = '' if len(text.split())<1 else text.split()[0]
                            prevTextData['y'] = element.y0
                            vLines = None
                    elif element.y1<50 and '№' in text:
                        texts = text.split()
                        footerPageNumber = '' if len(texts)<1 else texts[0]
                        footerBulletinNumber = '' if len(texts)<2 else texts[1].split('№')[1]
                        self.baseBulletin.setPageFooter(pageNum=pageNum, pageNumber=footerPageNumber,
                            bulletinNumber=footerBulletinNumber, x0=element.x0, y0=element.y0, x1=element.x1, y1=element.y1)



# Создание голосования и вывод данных

In [ ]:
#meeting = Meeting('Bulletin_Садовый_ул_Садовая,_29_meeting_No_1.pdf')
meeting = Meeting('помещение 1, ул. Садовая, 29 Садовый - собрание №1 - БЮЛЛЕТЕНЬ №6391122-3690641-5274319-1-71.pdf')
print(f'Имя базового PDF-файла: "{meeting.baseBulletin.fileName}"')
bulletin_number = meeting.baseBulletin.getBullotNumber()
print(f'Бюллетень № {bulletin_number.value}')
print(f'Координаты номера бюллетеня: cornerLD={bulletin_number.getCorners()[0]}, cornerRUp={bulletin_number.getCorners()[1]}')
meeting_data = meeting.getData()
for page_num, page_corners, page_images, page_markers_data, page_lines, footer in meeting_data:
    print(f'  Страница №{page_num +1}')
    print(f'    cornerLD={page_corners[0]}, cornerRUp={page_corners[1]}')
    # for i, image_data in enumerate(page_images):
    #   print(f'      image №{i+1}: name={image_data[0]} type={type(image_data[1])}')
    #   plt_img_show(image_data[1])
    for i, page_marker_data in enumerate(page_markers_data):
        marker = page_marker_data[1]
        print(f'      Маркер №{i+1}: name={marker.name}, cornerLD={marker.getCorners()[0]}, cornerRUp={marker.getCorners()[1]}')
        plt_img_show(page_marker_data[2])
    for i, page_line in enumerate(page_lines):
        print(f'      Вопрос {i+1}: №{page_line[0]}')
        for j, word_data in page_line[1].items():
            print(f'        Вариант {j+1}: "{word_data.word}", cornerLD={word_data.getCorners()[0]}, cornerRUp={word_data.getCorners()[1]}')
    print(f'    Нижний колонтитул: номер страницы: {footer.pageNumber}, бюллетень № {footer.bulletinNumber}, cornerLD={footer.getCorners()[0]}, cornerRUp={footer.getCorners()[1]}')

Имя базового PDF-файла: "помещение 1, ул. Садовая, 29 Садовый - собрание №1 - БЮЛЛЕТЕНЬ №6391122-3690641-5274319-1-71.pdf"
Бюллетень № 6391122-3690641-5274319/1-71
Координаты номера бюллетеня: cornerLD=[42.52, 775.79], cornerRUp=[427.66, 792.29]
  Страница №1
    cornerLD=[0, 0], cornerRUp=[595.28, 841.89]
      Маркер №1: name=I3, cornerLD=[42.52, 808.88], cornerRUp=[50.02, 816.38]


      Маркер №2: name=I4, cornerLD=[545.26, 808.88], cornerRUp=[552.76, 816.38]


      Маркер №3: name=I4, cornerLD=[42.52, 25.51], cornerRUp=[50.02, 33.01]


      Маркер №4: name=I4, cornerLD=[545.26, 25.51], cornerRUp=[552.76, 33.01]


      Вопрос 1: №1.1
        Вариант 1: "ЗА", cornerLD=[50.48, 463.52], cornerRUp=[201.62, 448.61]
        Вариант 2: "ПРОТИВ", cornerLD=[201.62, 463.52], cornerRUp=[379.31, 448.61]
        Вариант 3: "ВОЗДЕРЖАЛСЯ", cornerLD=[379.31, 463.52], cornerRUp=[510.23, 448.61]
      Вопрос 2: №2.1
        Вариант 1: "ЗА", cornerLD=[50.48, 516.28], cornerRUp=[201.62, 501.36]
        Вариант 2: "ПРОТИВ", cornerLD=[201.62, 516.28], cornerRUp=[379.31, 501.36]
        Вариант 3: "ВОЗДЕРЖАЛСЯ", cornerLD=[379.31, 516.28], cornerRUp=[510.23, 501.36]
      Вопрос 3: №3.
        Вариант 1: "ЗА", cornerLD=[50.48, 556.8], cornerRUp=[201.62, 541.89]
        Вариант 2: "ПРОТИВ", cornerLD=[201.62, 556.8], cornerRUp=[379.31, 541.89]
        Вариант 3: "ВОЗДЕРЖАЛСЯ", cornerLD=[379.31, 556.8], cornerRUp=[510.23, 541.89]
      Вопрос 4: №4.
        Вариант 1: "ЗА", cornerLD=[50.48, 634.01], cornerRUp=[201.62, 619.1]
        Вариант 2: "ПРОТИВ", cornerLD=[201.62, 634.01], cornerRUp=[379.31, 619.1]
        Вариант

...

      Вопрос 1: №6.
        Вариант 1: "ЗА", cornerLD=[50.48, 60.57], cornerRUp=[201.66, 45.67]
        Вариант 2: "ПРОТИВ", cornerLD=[201.66, 60.57], cornerRUp=[379.38, 45.67]
        Вариант 3: "ВОЗДЕРЖАЛСЯ", cornerLD=[379.38, 60.57], cornerRUp=[510.25, 45.67]
      Вопрос 2: №7.
        Вариант 1: "ЗА", cornerLD=[50.48, 113.32], cornerRUp=[201.62, 98.4]
        Вариант 2: "ПРОТИВ", cornerLD=[201.62, 113.32], cornerRUp=[379.31, 98.4]
        Вариант 3: "ВОЗДЕРЖАЛСЯ", cornerLD=[379.31, 113.32], cornerRUp=[510.23, 98.4]
      Вопрос 3: №8.
        Вариант 1: "ЗА", cornerLD=[50.48, 178.3], cornerRUp=[201.62, 163.39]
        Вариант 2: "ПРОТИВ", cornerLD=[201.62, 178.3], cornerRUp=[379.31, 163.39]
        Вариант 3: "ВОЗДЕРЖАЛСЯ", cornerLD=[379.31, 178.3], cornerRUp=[510.23, 163.39]
      Вопрос 4: №9.
        Вариант 1: "ЗА", cornerLD=[50.48, 243.28], cornerRUp=[201.62, 228.37]
        Вариант 2: "ПРОТИВ", cornerLD=[201.62, 243.28], cornerRUp=[379.31, 228.37]
        Вариант 3: "ВОЗДЕРЖ

      Маркер №2: name=I4, cornerLD=[545.26, 808.88], cornerRUp=[552.76, 816.38]


      Маркер №3: name=I4, cornerLD=[42.52, 25.51], cornerRUp=[50.02, 33.01]


      Маркер №4: name=I4, cornerLD=[545.26, 25.51], cornerRUp=[552.76, 33.01]


    Нижний колонтитул: номер страницы: 3, бюллетень № 6391122-3690641-5274319/1-71, cornerLD=[164.98, 26.81], cornerRUp=[430.3, 37.81]
